<a href="https://colab.research.google.com/github/iyogita/ChatBot_Budget/blob/main/AI_ML_NAGP_3176076_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI and ML Assignment for Budget :

# Resources Loading

In [1]:
!pip install pdfplumber
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers

from google.colab import files
import pdfplumber
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline, set_seed
from collections import deque
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 606.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Upload file
uploaded = files.upload()
pdf_path = 'Assignment Support Document.pdf'

Saving Assignment Support Document.pdf to Assignment Support Document.pdf


# Extract Text from PDFs

In [3]:
# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text

In [6]:
# Testing PDF text extaction
def test_pdf_extraction():
  pdf_text = extract_text_from_pdf(pdf_path)
  assert isinstance(pdf_text, str), "Extracted text should be a string"
  assert len(pdf_text) > 0, "Extracted text should not be empty"
  print("PDF extraction test passed.\n")
  display(pdf_text)


In [7]:
test_pdf_extraction();

PDF extraction test passed.



"GOVERNMENT OF INDIA\nINTERIM BUDGET 2024-2025\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1, 2024\n\nCONTENTS\nPART – A\nPage No.\nIntroduction 1\nInclusive Development and Growth 2\nSocial Justice 3\nExemplary Track Record of Governance,\nDevelopment and Performance (GDP) 7\nEconomic Management 8\nGlobal Context 9\nVision for ‘Viksit Bharat’ 10\nStrategy for ‘Amrit Kaal’ 11\nInfrastructure Development 17\nAmrit Kaal as Kartavya Kaal 22\nRevised Estimates 2023-24 23\nBudget Estimates 2024-25 23\nPART – B\nDirect taxes 25\nIndirect Taxes 26\nEconomy – Then and Now 28\n\nInterim Budget 2024-2025\nSpeech of\nNirmala Sitharaman\nMinister of Finance\nFebruary 1, 2024\nHon’ble Speaker,\nI present the Interim Budget for 2024-25.\nIntroduction\n1. The Indian economy has witnessed profound positive\ntransformation in the last ten years. The people of India are\nlooking ahead to the future with hope and optimism.\n2. With the blessings of the people, when our Government\nunde

# Segment Text and Generate Embeddings

In [8]:
# Segment text into chunks
def segment_text(text, chunk_size=512):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Generate embeddings for text chunks
def generate_embeddings(text_chunks):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    return model.encode(text_chunks, convert_to_tensor=True, normalize_embeddings=True)

In [12]:
# Testing embedding generation
def test_embedding_generation():
  text_chunks = [pdf_path]
  embeddings = generate_embeddings(text_chunks)
  assert embeddings.shape[0] == 1, "Should generate one embedding for one sentence"
  assert embeddings.shape[1] > 0, "Embedding size should be greater than zero"
  print("Embedding generation test passed.\n")
  display(embeddings)


In [13]:
test_embedding_generation();

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding generation test passed.



tensor([[-8.3239e-02,  3.2205e-02, -9.3686e-02, -9.3118e-03, -7.7991e-03,
          4.5528e-02, -9.2911e-03,  2.4877e-02, -1.0403e-01,  3.3580e-03,
         -2.3530e-02,  7.7437e-02,  2.9380e-02,  9.1991e-02, -4.5753e-02,
         -7.6813e-03, -5.8144e-02, -2.3628e-02,  2.8001e-02,  5.0193e-02,
          2.1056e-02,  2.1335e-02,  9.6805e-02,  1.1619e-02, -1.1593e-02,
         -3.9577e-03, -6.3501e-02, -7.1576e-02, -1.2930e-02, -4.4897e-02,
          3.6357e-02, -3.9580e-03,  4.8571e-02, -1.5642e-02,  7.9730e-02,
          3.6427e-02, -1.2721e-02, -1.2278e-02,  2.6487e-02,  4.5433e-02,
         -4.1653e-02, -2.1516e-02, -1.3998e-02,  9.9522e-03,  4.3549e-02,
          2.9174e-02, -2.2131e-03, -3.1950e-02,  6.7479e-03,  2.1439e-02,
         -1.0614e-02, -7.6207e-03, -3.8617e-02,  3.4686e-02,  2.9628e-02,
          3.4625e-02,  9.2031e-02,  6.5416e-03, -5.3048e-02, -7.1754e-02,
         -2.9301e-02,  4.4961e-02, -2.5103e-03,  8.3009e-02,  1.1621e-02,
          5.3347e-03, -1.9440e-02,  6.

# Utilize Vector Database (FAISS)

In [14]:
# Build FAISS index
def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index

In [15]:
# Search FAISS index
def search_index(index, query_embedding, k=5):
    query_embedding = np.array(query_embedding).reshape(1, -1).astype('float32')
    D, I = index.search(query_embedding, k)
    return I


In [16]:
def test_faiss_index_building():
    sample_texts = ["This is the first test sentence.", "This is the second test sentence."]
    embeddings = generate_embeddings(sample_texts)
    index = build_faiss_index(embeddings)
    assert index.ntotal == len(sample_texts), "Index size should match the number of text chunks"
    print("FAISS index building test passed.")


In [17]:
test_faiss_index_building();

FAISS index building test passed.


# Integrate GPT Model

In [18]:
# User context for refining queries
class UserContext:
    def __init__(self, max_size=5):
        self.context = deque(maxlen=max_size)

    def add_interaction(self, user_input, bot_response):
        self.context.append({'user_input': user_input, 'bot_response': bot_response})

    def get_context(self):
        return list(self.context)

    def check_context_size(self):
        return len(self.context)

In [19]:
# Refine query using GPT
def refine_query_with_gpt(user_context, query):
    set_seed(42)  # For reproducibility
    context = user_context.get_context()
    context_text = "\n".join([f"User: {interaction['user_input']}\nBot: {interaction['bot_response']}" for interaction in context])
    prompt = f"{context_text}\nUser: {query}\nBot:"
    response = generator(prompt, max_length=150, num_return_sequences=1)[0]['generated_text']
    refined_response = response[len(prompt):].strip()
    return refined_response


In [20]:
def test_query_refinement():
    user_context = UserContext(max_size=5)
    user_context.add_interaction("What is AI?", "AI stands for Artificial Intelligence.")
    query = "Explain AI."
    refined_query = refine_query_with_gpt(user_context, query)
    assert isinstance(refined_query, str), "Refined query should be a string"
    assert len(refined_query) > 0, "Refined query should not be empty"
    print("Query refinement test passed.")


In [22]:
# Initialize GPT-2 generator
generator = pipeline('text-generation', model='gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [23]:
test_query_refinement();

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query refinement test passed.


In [24]:
# Example usage
pdf_text = extract_text_from_pdf(pdf_path)
text_chunks = segment_text(pdf_text)
embeddings = generate_embeddings(text_chunks)
index = build_faiss_index(embeddings)

In [25]:
# Simulate user interactions
user_context = UserContext()
user_context.add_interaction("What is AI?", "AI stands for Artificial Intelligence.")
user_context.add_interaction("What is ML?", "ML stands for Machine Learning.")
user_context.add_interaction("What is NLP?", "NLP stands for Natural Language Processing.")


In [26]:
# Refine a user query
query = "How does the interim budget affect the economy explain?"
refined_response = refine_query_with_gpt(user_context, query)
print(refined_response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The interim budget has been estimated to help reduce net debt for the US government with a 2% increase in the national debt and to help reduce US debt burden in the longer term. The interim budget also includes an investment in infrastructure to create jobs and improve the country's reputation within the world of finance.
User: What is the best way to improve the country?
Bot: We're trying to add to our debt


# Performance

In [27]:
def performance_evaluation(pdf_path, query_text):
    # Text Extraction Performance
    start_time = time.time()
    pdf_text = extract_text_from_pdf(pdf_path)
    end_time = time.time()
    extraction_time = end_time - start_time

    # Text Segmentation Performance
    start_time = time.time()
    text_chunks = segment_text(pdf_text)
    end_time = time.time()
    segmentation_time = end_time - start_time
    chunk_count = len(text_chunks)

    # Embedding Generation Performance
    start_time = time.time()
    embeddings = generate_embeddings(text_chunks)
    end_time = time.time()
    embedding_generation_time = end_time - start_time
    embedding_size = embeddings.shape[1]

    # FAISS Index Building Performance
    start_time = time.time()
    index = build_faiss_index(embeddings)
    end_time = time.time()
    index_building_time = end_time - start_time
    index_size = index.ntotal

    # Query Response Performance
    query_embedding = generate_embeddings([query_text])[0]
    start_time = time.time()
    results = search_index(index, query_embedding)
    end_time = time.time()
    query_time = end_time - start_time

    # Overall System Performance
    total_start_time = time.time()
    # Complete workflow
    pdf_text = extract_text_from_pdf(pdf_path)
    text_chunks = segment_text(pdf_text)
    embeddings = generate_embeddings(text_chunks)
    index = build_faiss_index(embeddings)
    query_embedding = generate_embeddings([query_text])[0]
    results = search_index(index, query_embedding)
    total_end_time = time.time()
    total_processing_time = total_end_time - total_start_time

    # Report
    print(f"Text Extraction Time: {extraction_time} seconds")
    print(f"Text Segmentation Time: {segmentation_time} seconds")
    print(f"Number of Chunks: {chunk_count}")
    print(f"Embedding Generation Time: {embedding_generation_time} seconds")
    print(f"Embedding Size: {embedding_size}")
    print(f"Index Building Time: {index_building_time} seconds")
    print(f"Index Size: {index_size}")
    print(f"Query Response Time: {query_time} seconds")
    print(f"Total Processing Time: {total_processing_time} seconds")

# Example usage
performance_evaluation(pdf_path, "What is the budget impact on middle class?")

Text Extraction Time: 2.8330190181732178 seconds
Text Segmentation Time: 0.0007784366607666016 seconds
Number of Chunks: 11
Embedding Generation Time: 2.0464117527008057 seconds
Embedding Size: 384
Index Building Time: 0.000102996826171875 seconds
Index Size: 11
Query Response Time: 0.0002276897430419922 seconds
Total Processing Time: 5.056834697723389 seconds


# Performance

**Optimize Embedding Generation:**

Use batch processing to generate embeddings for text chunks in parallel. This can significantly reduce the time taken for embedding generation.

**Efficient Indexing:**

Ensure that the FAISS index is built and searched efficiently. Consider using a more advanced FAISS index type if the default index is not performant enough.

**Caching:**

Implement caching mechanisms to store frequently queried results and avoid redundant computations.

# Scalability

**Concurrent User Queries:**
Implement concurrency handling to manage multiple users querying the system simultaneously. This can be achieved using asynchronous programming or multi-threading.

**Large PDFs:**
Implement a mechanism to handle large PDFs efficiently. This might involve processing the PDF in chunks and using a queue to manage large-scale text extraction and embedding generation.

# Security

**Data Encryption:**

Encrypt sensitive data to protect it from unauthorized access. Ensure that the PDF files and embeddings are stored securely.

**Access Control:**

Implement authentication and authorization mechanisms to restrict access to the system.

**Input Validation:**

Validate user inputs to prevent injection attacks or other malicious inputs.

# Context Verify

In [28]:
def test_context_preservation():
    user_context = UserContext(max_size=5)

    # Simulate adding more than 5 interactions
    for i in range(1, 7):
        user_context.add_interaction(f"Question {i}", f"Answer {i}")

    # Check the size of the context
    assert len(user_context.get_context()) == 5, "Context size should be 5"

    # Check the content of the context
    expected_context = [
        {"user_input": "Question 2", "bot_response": "Answer 2"},
        {"user_input": "Question 3", "bot_response": "Answer 3"},
        {"user_input": "Question 4", "bot_response": "Answer 4"},
        {"user_input": "Question 5", "bot_response": "Answer 5"},
        {"user_input": "Question 6", "bot_response": "Answer 6"}
    ]

    assert user_context.get_context() == expected_context, "Context is not preserved correctly"
    print("Context preservation test passed.")


In [29]:
test_context_preservation()

Context preservation test passed.
